Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50051)
clean_all_dbs(speos.client)

New project

In [ ]:
p = script.Project(speos=speos)
print(p)

# Create
Create locally  
The mention "local: " is added when printing the optical property

In [ ]:
op1 = p.create_optical_property(name="Material.1")
op1.set_surface_mirror(reflectance=80)\
   .set_volume_opaque()\
   .set_geometries(geometries=[script.GeoRef.from_native_link(geopath="TheBodyB"),\
                               script.GeoRef.from_native_link(geopath="TheBodyC")])
print(op1)

Push it to the server.  
Now that it is committed to the server, the mention "local: " is no more present when printing the optical property.  

In [ ]:
op1.commit()
print(op1)

Another example by setting several characteristics

In [ ]:
op2 = p.create_optical_property(name="Material.2")
op2.set_surface_opticalpolished()\
   .set_volume_library(path=os.path.join(tests_data_path, "AIR.material"))\
   .set_geometries(geometries=[script.GeoRef.from_native_link(geopath="TheBodyB"),\
                               script.GeoRef.from_native_link(geopath="TheBodyC")])\
   .commit()
print(op2)

## Default values
Some default values are available when applicable in every methods and class.

In [ ]:
op3 = p.create_optical_property(name="Material.3").commit()
print(op3)

# Read
## Material Instance
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(op1)

## Project

In [ ]:
print(p)

# Update
Tipp: if you are manipulating an optical property already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
op1.set_volume_optic()\
   .set_surface_opticalpolished()\
   .commit()
print(op1)

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
op1.set_surface_mirror() # set surface as a mirror but no commit
op1.reset() # reset -> this will apply the server value to the local value
op1.delete() # delete (to display the local value with the below print)
print(op1)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
op2.delete()
print(op2)

In [ ]:
op1.delete()
op3.delete()

# Cleanup

In [ ]:
clean_all_dbs(speos.client)